In [ ]:
!pip install antigranular



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.

In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Heart Disease Prediction Hackathon")

Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: 38fa149e-242b-48eb-80aa-e99803f7e802, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
!pip install opacus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
%%ag
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_pandas import standard_scaler, PrivateDataFrame

from op_tensorflow import PrivateKerasModel, PrivateDataLoader
import opacus

from pandas import DataFrame


/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):



In [ ]:
%%ag
x_train = heart_disease_prediction_hackathon_dataset["train_x"]
y_train = heart_disease_prediction_hackathon_dataset["train_y"]
x_test = heart_disease_prediction_hackathon_dataset["test_x"]

In [ ]:
%%ag
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


In [ ]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=0.1)
x_test_scaled = standard_scaler(PrivateDataFrame(x_test), eps=0.1)


In [ ]:
%%ag
seqM = Sequential([
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1, activation='sigmoid')
])


In [ ]:
%%ag
optimizer = Adam(learning_rate=0.01)


In [ ]:
%%ag
dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1, noise_multiplier=1)

In [ ]:
%%ag

dp_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"])

In [ ]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_scaled, label_df=y_train, batch_size=128)

In [ ]:
%%ag
dp_model.fit(data_loader, epochs=10, target_delta=1e-5)

Epoch 1/10

63/63 - 3s - loss: 0.6104 - accuracy: 0.7120 - 3s/epoch - 53ms/step

Epoch 2/10

63/63 - 3s - loss: 0.6152 - accuracy: 0.7074 - 3s/epoch - 54ms/step

Epoch 3/10

63/63 - 3s - loss: 0.6185 - accuracy: 0.7051 - 3s/epoch - 52ms/step

Epoch 4/10

63/63 - 3s - loss: 0.6047 - accuracy: 0.7079 - 3s/epoch - 52ms/step

Epoch 5/10

63/63 - 3s - loss: 0.6090 - accuracy: 0.7166 - 3s/epoch - 52ms/step

Epoch 6/10

63/63 - 3s - loss: 0.6039 - accuracy: 0.7133 - 3s/epoch - 53ms/step

Epoch 7/10

63/63 - 3s - loss: 0.6162 - accuracy: 0.6997 - 3s/epoch - 54ms/step

Epoch 8/10

63/63 - 3s - loss: 0.6065 - accuracy: 0.7111 - 3s/epoch - 52ms/step

Epoch 9/10

63/63 - 3s - loss: 0.6051 - accuracy: 0.7102 - 3s/epoch - 52ms/step

Epoch 10/10

63/63 - 3s - loss: 0.6005 - accuracy: 0.7168 - 3s/epoch - 52ms/step

message: Error ID is - c08b6a80-8df9-4378-a37f-26ef39002e74 - Error from upstream service: Client error '400 Bad Request' for url 'http://supervisor-prod-private.antigranular.com/privacyReq

In [ ]:
%%ag
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=.1)

y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])

In [ ]:

%%ag
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

In [ ]:
%%ag
result = submit_predictions(y_pred)